In [1]:
# ライブラリのインポート
import numpy as np
import pandas as pd
import psycopg2
import datetime

In [2]:
dbname = "kddi_18_param_custom2"
npyname = "trkn_18"
datedir = "230120"
sttime = "2022-04-01 21:00:00"

In [3]:
#解析結果の読み込み
timeseries_sep = np.load("./" + datedir+'/'+ npyname + ".npy")

In [4]:
timeseries_sep.shape

(24, 207)

In [5]:
timeseries_sep

array([[ 88,  62,  95, ..., 109, 109,   1],
       [ 77,  74,  90, ..., 116, 116,   0],
       [ 64,  74,  86, ..., 106, 106,   0],
       ...,
       [ 13,   4,  24, ...,  26,  26,   0],
       [ 21,  12,  12, ...,  28,  28,   0],
       [ 16,   6,  11, ...,  23,  23,   0]], dtype=int64)

In [6]:
#メッシュ情報の読み込み
csv_file = r'./'+datedir+'/'+ npyname+'_header.csv'
df = pd.read_csv(csv_file, dtype={'date':str}, index_col=0, parse_dates=True)
df_i = df.loc["gridcode"] #gridcode #end_mesh_area

In [7]:
connection = psycopg2.connectconnection = psycopg2.connect(host="localhost", database="analysisdb", user="postgres",password="setouchi1311")

In [ ]:
dte = datetime.datetime.strptime(sttime, '%Y-%m-%d %H:%M:%S')

for lp in range(4):
    cur =connection.cursor()
    curs =connection.cursor()

    checkMeshtbranList = [] #確認対象メッシュのKDDI値

    #まずは時刻ごとのメッシュ一覧を取得
    sql = "select mesh from " + dbname + " WHERE times = '"+str(dte)+"'"
    cur.execute(sql)
    meshLists = cur.fetchall()
    
    print("time =" + str(dte) + " target =" + str(len(meshLists)))
    for mesh in meshLists:
        
        meshkey = str(int(mesh[0]))
        
        #トラカン値を持つメッシュならば解析しない
        find = 0
        for chkm in df_i.values:
            if str(int(chkm)) == meshkey:
                find = 1
                break
        if find == 1:
            continue

        sql = "select kddi_tran.tbran, T.gridcode,T.断面交通量,kddi_tran.mesh "
        sql += "from " + dbname + " AS kddi_tran "
        sql += "INNER JOIN pointdata as a ON kddi_tran.mesh = cast(a.grid_code AS BIGINT) "
        sql += "CROSS JOIN LATERAL "
        sql += "("
        sql += "SELECT 時刻, gridcode , 断面交通量 from trakan2204_z3out AS trakan_tran "
        sql += "inner JOIN pointdata as b ON trakan_tran.gridcode = cast(grid_code AS BIGINT) "
        sql += "WHERE 時刻 = '"+str(dte)+"' "
        sql += "ORDER BY a.geom <-> b.geom, ST_Distance(a.geom, b.geom) "
        sql += "LIMIT 1) AS T "
        sql += "WHERE times = '"+str(dte)+"' and kddi_tran.mesh = " + meshkey
        curs.execute(sql)
        datalists = curs.fetchall()
        #print(datalists) #(88.0, 50324526133, 13) #対象MESH値、確認対象メッシュ、対象の断面交通量
        
        if len(datalists) == 0:
            #print(sql)
            continue

        print(meshkey)
        #確認対象メッシュの kddi_tran.tbran値
        enabledcheck = meshkey in checkMeshtbranList
        baseMeshItem = 0
        if enabledcheck == True:
            for meshitem in checkMeshtbranList:
                if str(int(mesh)) == meshitem[0]:
                    baseMeshItem = meshitem[1]
                    break
        else:
            cur3 =connection.cursor()
            sql = "select kddi_tran.tbran from " + dbname + " AS kddi_tran "
            sql += "WHERE times = '"+str(dte)+"' and kddi_tran.mesh = " + str(datalists[0][1])
            cur3.execute(sql)
            baseMeshParam = cur3.fetchall()
            setparam = 0
            if len(baseMeshParam) != 0:
                setparam = baseMeshParam[0][0]
            if len(checkMeshtbranList) == 0:
                checkMeshtbranList = [meshkey , setparam]
            else:
                checkMeshtbranList.append([meshkey , setparam])
            cur3.close()
            baseMeshItem = setparam

        #メッシュ値を算出
        if baseMeshItem == 0:
            param = 0
        else:
            param = datalists[0][0] / baseMeshItem * datalists[0][2]
        print(str(datalists[0][0]) + ":" +  str(baseMeshItem) + ":" +  str( datalists[0][2]) )
            
        sql = "UPDATE " + dbname + " set tbran = "+ str(param) + " where times = '"  + str(dte) + "' and mesh = " + meshkey + ";"
        #print(sql)
        cur.execute(sql)
        connection.commit()
    cur.close()
    curs.close()
    dte = dte + datetime.timedelta(minutes=15)
connection.commit()

time =2022-04-01 21:00:00 target =6681
50325575224
0.003060799:174.0:4
50325575241
0.0058171544:174.0:4
50325575242
0.052837044:174.0:4
50325575243
0.0007917418:174.0:4
50325575244
0.0009853588:174.0:4
50325575421
0.00012001395:174.0:4
50325575422
0.0010936344:174.0:4
50325575423
0.0012303214:174.0:4
50325575424
0.0119465105:174.0:4
50325575442
0.0012130756:174.0:4
50325575444
0.0129642105:174.0:4
50325576111
9.60622e-05:174.0:4
50325576112
0.031177267:174.0:4
50325576121
0.022442995:101.0:9
50325576122
0.009868199:101.0:9
50325576123
0.018008135:101.0:9
50325576124
0.0009402514:101.0:9
50325576131
0.013627105:174.0:4
50325576132
0.0065905796:174.0:4
50325576133
0.001509056:174.0:4
50325576141
0.043626208:174.0:4
50325576142
0.030678805:101.0:9
50325576143
0.09503723:174.0:4
50325576144
0.013120151:174.0:4
50325576211
0.06044194:101.0:9
50325576213
0.04449138:101.0:9
50325576214
0.015479006:101.0:9
50325576241
0.005336268:101.0:9
50325576243
0.022713967:101.0:9
50325576311
0.006739877:

0.008867217:173.0:28
50325630113
0.013551915:173.0:28
50325630123
0.00037593208:173.0:28
50325630131
0.0024778524:173.0:28
50325630134
0.036951877:173.0:28
50325630141
0.0011501807:173.0:28
50325630442
0.002043644:173.0:28
50325631341
0.0030019633:173.0:28
50325631342
0.014509368:173.0:28
50325631343
0.00017806492:173.0:28
50325632121
0.00028125802:102.0:7
50325633111
0.0108648855:224.0:31
50325633112
0.025632158:224.0:31
50325633113
0.020116247:224.0:31
50325633114
0.009297512:234.0:5
50325633121
0.008521905:234.0:5
50325633233
0.002366282:234.0:5
50325633234
0.0007812092:234.0:5
50325633241
0.0016363307:234.0:5
50325633244
0.0019129307:234.0:5
50325633314
0.005151423:174.0:28
50325633423
0.0038816582:174.0:28
50325633431
0.027990963:174.0:28
50325633432
0.010914187:174.0:28
50325633433
0.0028330777:174.0:28
50325633434
0.0011413359:174.0:28
50325633441
0.0052221315:174.0:28
50325634314
0.0023177648:234.0:5
50325634322
0.0074371304:234.0:5
50325635113
0.00048246732:234.0:5
50325635114

50325662214
0.009083112:119.0:22
50325662221
0.0018614233:113.0:56
50325662222
0.012845932:113.0:56
50325662322
0.007912295:45.0:65
50325662324
0.032604326:45.0:65
50325662332
0.04383324:45.0:65
50325662333
0.0015613427:0.0:15
50325662431
0.018834032:45.0:65
50325662432
0.0004906049:45.0:65
50325662434
0.010523016:45.0:65
50325662441
0.03130651:45.0:65
50325662442
0.096469894:45.0:65
50325662444
0.0010756478:45.0:65
50325663212
0.00785544:31.0:53
50325663221
0.0033685076:31.0:53
50325663223
0.00020733428:31.0:53
50325663233
0.029321704:31.0:53
50325663234
0.00653103:31.0:53
50325663333
0.06424128:45.0:65
50325663341
0.0249521:31.0:53
50325663342
0.0053714793:31.0:53
50325663344
0.0059941644:31.0:53
50325663411
0.002649467:31.0:53
50325663412
0.024989475:31.0:53
50325664124
0.016148575:53.0:44
50325664132
0.0018564168:31.0:53
50325664133
0.0076298066:31.0:53
50325664134
0.037665397:31.0:53
50325664142
0.008490299:53.0:44
50325664312
0.008290471:31.0:53
50325664313
0.0009286117:31.0:53
5

50325624323
0.03619895:234.0:5
50325624331
0.0005490486:234.0:5
50325624333
0.021368824:234.0:5
50325625121
0.011206007:234.0:5
50325625132
0.0002084875:234.0:5
50325625133
0.00047336332:234.0:5
50325625134
0.00083602685:234.0:5
50325625431
0.0002602865:256.0:22
50325625434
0.0004552696:256.0:22
50325625442
0.0063306233:256.0:22
50325625444
0.0054786997:256.0:22
50325626131
0.0045305784:256.0:22
50325626133
0.0012583045:256.0:22
50325627444
0.0044861287:256.0:22
50325628313
0.003163989:256.0:22
50325628333
0.0054744836:256.0:22
50325629421
0.00064870575:255.0:28
50325629422
0.0017385262:255.0:28
50325629442
0.005919989:255.0:28
50325630411
0.019675344:173.0:28
50325630414
0.0017906235:173.0:28
50325630421
0.054710254:173.0:28
50325630431
0.021204837:173.0:28
50325630432
0.0028558078:173.0:28
50325630433
0.007916413:173.0:28
50325632222
0.008717224:224.0:31
50325632224
0.0008914871:102.0:7
50325632244
0.0018461733:102.0:7
50325632322
0.00039361836:102.0:7
50325632323
0.003544244:102.0:7

50325661244
0.00054729916:0.0:15
50325661311
0.0040317527:0.0:15
50325661312
0.048212044:0.0:15
50325661313
0.004050625:0.0:15
50325661341
0.0022144057:0.0:15
50325661342
0.030904345:0.0:15
50325661343
0.017335538:0.0:15
50325661344
0.022628866:0.0:15
50325661411
0.008641553:0.0:15
50325661413
0.014957193:0.0:15
50325661414
0.018157642:0.0:15
50325662131
0.00100428:119.0:22
50325662133
0.07281302:119.0:22
50325662134
0.0053541595:119.0:22
50325662141
0.049731802:119.0:22
50325662143
0.008139484:119.0:22
50325662144
0.020585569:119.0:22
50325662242
0.0012829669:113.0:56
50325662243
0.010765825:45.0:65
50325662311
0.010493242:119.0:22
50325662312
0.054799262:119.0:22
50325662313
0.0031616995:119.0:22
50325662314
0.003424503:119.0:22
50325662412
0.0044510774:45.0:65
50325662414
0.009040383:45.0:65
50325662421
0.0036396612:45.0:65
50325662422
0.015372487:45.0:65
50325662423
0.09648321:45.0:65
50325662424
0.016403805:45.0:65
50325663112
0.0020805984:113.0:56
50325663121
0.08279328:113.0:56


...

0.0064127897:183.0:16
50325751241
0.0075715855:183.0:16
50325751243
0.0027182456:183.0:16
50325751314
0.012548037:152.0:12
50325752111
0.012418371:183.0:16
50325752113
0.03330105:183.0:16
50325752121
0.04319955:127.0:24
50325752122
0.00080003263:127.0:24
50325752314
0.006631482:183.0:16
50325752321
0.00031583733:183.0:16
50325752323
0.0034304364:183.0:16
50325752324
0.004325998:183.0:16
50325752331
0.030008778:183.0:16
50325752332
0.0034469115:183.0:16
50325752413
0.0063134525:67.0:2
50325752414
0.0024369203:67.0:2
50325752422
0.020481415:127.0:24
50325752423
0.012692053:67.0:2
50325752424
0.079082444:67.0:2
50325753143
0.029744945:127.0:24
50325753144
0.004078951:127.0:24
50325753213
0.0016587507:127.0:24
50325753241
0.011295004:127.0:24
50325753242
0.0060323337:127.0:24
50325753244
0.02284044:127.0:24
50325753333
0.0025246092:67.0:2
50325753341
0.0017515197:67.0:2
50325753342
0.015491795:67.0:2
50325753343
0.0012042671:67.0:2
50325753344
0.0028931005:67.0:2
50325760111
0.020698123:15

0.020886078:98.0:54
50325682121
0.024880972:98.0:54
50325682122
0.0028654635:98.0:54
50325682123
0.0043891743:98.0:54
50325682124
0.07515944:98.0:54
50325682223
0.051886287:28.0:60
50325682224
0.024132045:28.0:60
50325682231
0.011894562:98.0:54
50325682232
0.0057414174:28.0:60
50325682233
0.037525304:119.0:76
50325682241
0.0007064929:28.0:60
50325682242
0.020671224:28.0:60
50325682333
0.03323517:119.0:76
50325682334
0.026216745:119.0:76
50325682341
0.01874523:119.0:76
50325682342
0.03687567:119.0:76
50325682344
0.084710985:119.0:76
50325682424
0.011822695:158.0:60
50325682431
0.008349133:158.0:60
50325682432
2.3113447e-05:158.0:60
50325682433
0.04899699:158.0:60
50325682434
0.05281833:158.0:60
50325682441
0.029824415:158.0:60
50325683213
0.0070852907:23.0:21
50325683214
0.0016763732:103.0:19
50325683221
0.011285638:103.0:19
50325683222
0.038237333:103.0:19
50325683244
0.0037169312:199.0:20
50325683311
0.025538117:28.0:60
50325683312
0.0025435798:28.0:60
50325683313
0.03745901:28.0:60
5

50325772241
0.0130257495:67.0:2
50325772243
0.005021657:67.0:2
50325772313
0.00890135:183.0:16
50326505413
0.0011617192:73.0:32
50326505444
0.018421384:73.0:32
50326506114
0.00025163527:80.0:8
50326506234
0.0030926047:251.0:14
50326506241
0.001351356:251.0:14
50326506243
0.0063890754:251.0:14
50326506321
0.031230975:73.0:32
50326506411
0.0025852048:251.0:14
50326506412
0.026850637:251.0:14
50326507131
0.0012656243:95.0:12
50326507142
0.023184907:95.0:12
50326507211
0.005097736:80.0:20
50326507214
0.004103953:95.0:12
50326507423
0.0067875534:95.0:12
50326507424
0.018110357:95.0:12
50326507434
0.023031427:95.0:12
50326507441
0.0010592751:95.0:12
50326507442
8.258736e-06:95.0:12
50326508123
0.026409382:107.0:19
50326508124
0.041526623:107.0:19
50326508132
0.0017358065:107.0:19
50326508141
0.009177998:107.0:19
50326508223
0.032208063:107.0:19
50326508231
0.009232817:107.0:19
50326508232
0.031671304:107.0:19
50326508234
0.037547447:107.0:19
50326508321
0.044906534:107.0:19
50326508323
0.022

50326578411
0.03506113:108.0:11
50326578413
0.041118346:108.0:11
50326578421
0.0031750433:108.0:11
50326578422
0.0009467043:108.0:11
50326578423
0.0016276329:108.0:11
50326578424
0.010439074:108.0:11
50326579232
0.0074484916:108.0:11
50326579234
0.021812502:108.0:11
50326579241
0.00068504177:108.0:11
50326579243
0.013559801:108.0:11
50326579244
0.051739343:108.0:11
50326584142
0.0060598673:124.0:19
50326584211
0.0021862695:124.0:19
50326584231
0.009093535:124.0:19
50326584233
0.008968867:124.0:19
50326584241
0.002144716:124.0:19
50326585133
0.0017144722:124.0:19
50326588244
0.0017093015:108.0:11
50326589111
0.0011280642:108.0:11
50326589112
0.014813706:108.0:11
50326589113
0.003258449:108.0:11
50326589114
0.0039610397:108.0:11
50326589121
0.024319075:108.0:11
50326589122
0.0428209:108.0:11
50326589442
0.037501857:157.0:24
50326593223
0.00397887:124.0:19
50326594142
0.0027779844:124.0:19
50326595342
0.0033201887:124.0:19
50326600111
0.006431397:123.0:16
50326600222
0.063750565:128.0:11


50326518423
0.057643004:13.0:16
50326518424
0.01548075:13.0:16
50326518432
0.06043066:13.0:16
50326519234
0.0048010377:32.0:45
50326519242
0.014795953:32.0:45
50326519323
0.031757537:0.0:47
50326519333
0.007230385:13.0:16
50326519334
0.0028937198:145.0:14
50326519341
7.235678e-05:0.0:47
50326519342
0.034300476:145.0:14
50326519441
0.032465085:145.0:14
50326525212
0.0019653146:73.0:32
50326525214
0.0019189307:73.0:32
50326526444
0.0010994731:82.0:24
50326527111
0.0012623221:228.0:10
50326527112
0.008062701:228.0:10
50326527114
0.07112156:228.0:10
50326527121
0.00642013:228.0:10
50326527134
0.02589475:228.0:10
50326527141
0.0019685458:228.0:10
50326527142
0.007541135:228.0:10
50326527144
0.035897948:228.0:10
50326527212
0.0008332031:23.0:8
50326527314
0.06433749:82.0:24
50326527321
0.007858204:228.0:10
50326527322
0.011781676:228.0:10
50326527323
0.000558028:228.0:10
50326527411
0.032978117:23.0:8
50326527412
0.0040436136:23.0:8
50326527414
0.009751411:23.0:8
50326527421
0.032949112:23.0

In [ ]:
#ConvertMeshtoRoad

In [ ]:
connection.close()